# CUVS Avanzado: Algoritmos IVF-PQ y CAGRA con Optimización de Parámetros

Este notebook explora características avanzadas de CUVS, incluyendo algoritmos de última generación y técnicas de optimización.

## Algoritmos Avanzados
- **IVF-PQ**: Indexación con cuantización producto, optimiza memoria y velocidad
- **CAGRA**: Graph-based search, máxima velocidad para aplicaciones en tiempo real

## Aplicaciones Avanzadas
- **Búsqueda en tiempo real**: Chatbots, motores de recomendación
- **Datasets masivos**: Billones de vectores con compresión
- **Optimización de recursos**: Balance memoria vs velocidad vs precisión

## Comparaciones de Rendimiento
Compararemos IVF-PQ vs CAGRA vs IVF-Flat en términos de:
- Velocidad de construcción y búsqueda
- Uso de memoria
- Precisión (recall)
- Escalabilidad

## 1. Instalar y Configurar CUVS Avanzado

Asegúrate de tener CUVS instalado con soporte completo para algoritmos avanzados.

### Algoritmos Disponibles
- **IVF-Flat**: Baseline, balance velocidad/precisión
- **IVF-PQ**: Compresión con cuantización producto
- **CAGRA**: Búsqueda basada en grafos, máxima velocidad
- **Brute Force**: Referencia exacta

### Cuándo Usar Cada Uno
- **IVF-Flat**: Desarrollo, datasets medianos, precisión crítica
- **IVF-PQ**: Producción, datasets grandes, optimización memoria
- **CAGRA**: Tiempo real, alta throughput, tolerancia a recall moderado

In [ ]:
!pip install cuvs-cu12 --extra-index-url=https://pypi.nvidia.com

import numpy as np
import cupy as cp
from cuvs.common import Resources
from cuvs.neighbors import ivf_pq, cagra
import time
import matplotlib.pyplot as plt

## 2. Preparar Datos para Algoritmos Avanzados

Los algoritmos avanzados requieren consideraciones especiales en preparación de datos.

### Requisitos por Algoritmo
- **IVF-PQ**: Datos deben ser float32, dimensionalidad compatible con subvectores
- **CAGRA**: Optimizado para datos normalizados, mejor con cosine similarity
- **Todos**: Benefician de normalización y preprocesamiento

### Optimizaciones de Datos
- **Normalización**: Importante para métricas de similitud
- **Dimensionalidad**: 128-1024 típico, más alto para mejor precisión
- **Formato**: CuPy arrays para máxima velocidad GPU

In [ ]:
np.random.seed(42)
n_samples = 50000
dim = 128
dataset = np.random.randn(n_samples, dim).astype(np.float32)
dataset = dataset / np.linalg.norm(dataset, axis=1, keepdims=True)

n_queries = 1000
queries = np.random.randn(n_queries, dim).astype(np.float32)
queries = queries / np.linalg.norm(queries, axis=1, keepdims=True)

print(f"Dataset: {dataset.shape}, Queries: {queries.shape}")

## 3. Construir Índice IVF-PQ

IVF-PQ combina clustering IVF con cuantización producto para compresión extrema.

### Parámetros Clave
- `n_lists`: Clusters (igual que IVF-Flat)
- `n_subquantizers`: Subvectores para cuantización (típicamente 8-32)
- `n_bits`: Bits por subvector (8 recomendado)
- `pq_search`: Parámetros de búsqueda específicos de PQ

### Ventajas
- **Compresión 8-16x**: Reduce memoria significativamente
- **Velocidad**: Búsqueda rápida en datasets comprimidos
- **Escalabilidad**: Maneja billones de vectores

### Desventajas
- **Precisión**: Ligeramente menor que IVF-Flat
- **Construcción**: Más lenta que IVF-Flat

In [ ]:
resources = Resources()

# IVF-PQ
pq_dim = 64
build_params_ivf_pq = ivf_pq.IndexParams(
    n_lists=1024, 
    metric="cosine", 
    pq_dim=pq_dim, 
    pq_bits=8
)
start = time.time()
index_ivf_pq = ivf_pq.build(build_params_ivf_pq, cp.asarray(dataset), resources=resources)
resources.sync()
ivf_pq_build_time = time.time() - start
print(f"IVF-PQ index built in {ivf_pq_build_time:.2f}s")

# CAGRA
build_params_cagra = cagra.IndexParams(metric="cosine", intermediate_graph_degree=64, graph_degree=32)
start = time.time()
index_cagra = cagra.build(build_params_cagra, cp.asarray(dataset), resources=resources)
resources.sync()
cagra_build_time = time.time() - start
print(f"CAGRA index built in {cagra_build_time:.2f}s")

## 4. Construir Índice CAGRA

CAGRA (Compressed Approximate Graph for Rapid ANN) es el algoritmo más rápido de CUVS.

### Cómo Funciona
- Construye un grafo de navegación aproximado
- Usa compresión y optimizaciones GPU avanzadas
- Optimizado para búsqueda en tiempo real

### Parámetros Clave
- `build_algo`: Algoritmo de construcción ("ivf_pq" o "nn_descent")
- `search_width`: Ancho de búsqueda durante construcción
- `itopk_size`: Tamaño del conjunto candidato

### Ventajas
- **Velocidad extrema**: 10-100x más rápido que IVF-Flat
- **Escalabilidad**: Excelente para datasets masivos
- **Adaptable**: Funciona bien con diferentes métricas

### Desventajas
- **Construcción lenta**: Más tiempo que otros algoritmos
- **Memoria**: Requiere más memoria que IVF-PQ

In [ ]:
k = 10

# IVF-PQ search
search_params_ivf_pq = ivf_pq.SearchParams(n_probes=20)
start = time.time()
dist_ivf_pq, neigh_ivf_pq = ivf_pq.search(search_params_ivf_pq, index_ivf_pq, cp.asarray(queries), k=k, resources=resources)
resources.sync()
ivf_pq_search_time = time.time() - start

# CAGRA search
search_params_cagra = cagra.SearchParams(itopk_size=64)
start = time.time()
dist_cagra, neigh_cagra = cagra.search(search_params_cagra, index_cagra, cp.asarray(queries), k=k, resources=resources)
resources.sync()
cagra_search_time = time.time() - start

print(f"IVF-PQ search: {ivf_pq_search_time:.4f}s")
print(f"CAGRA search: {cagra_search_time:.4f}s")

## 5. Búsqueda con Algoritmos Avanzados

Cada algoritmo tiene parámetros de búsqueda específicos para optimizar rendimiento.

### Parámetros por Algoritmo
- **IVF-PQ**: `n_probes`, `lut_dtype` (tipo de tabla lookup)
- **CAGRA**: `itopk`, `search_width`, `max_iterations`
- **Comunes**: `k` (vecinos), `batch_size` (procesamiento por lotes)

### Optimización de Búsqueda
- **Batch processing**: Procesar múltiples queries juntas
- **Parámetros adaptativos**: Ajustar según requerimientos de latencia
- **Refinamiento**: Usar resultados aproximados como candidatos para búsqueda exacta

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
exact_distances = cosine_distances(queries, dataset)
exact_neighbors = np.argsort(exact_distances, axis=1)[:, :k]

def recall_at_k(pred, true, k):
    recall = 0
    for i in range(len(pred)):
        recall += len(set(pred[i]) & set(true[i])) / k
    return recall / len(pred)

recall_ivf_pq = recall_at_k(cp.asnumpy(neigh_ivf_pq), exact_neighbors, k)
recall_cagra = recall_at_k(cp.asnumpy(neigh_cagra), exact_neighbors, k)

print(f"IVF-PQ Recall@{k}: {recall_ivf_pq:.4f}")
print(f"CAGRA Recall@{k}: {recall_cagra:.4f}")

## 6. Evaluar y Comparar Algoritmos

Comparamos rendimiento y precisión de todos los algoritmos.

### Métricas de Comparación
- **Recall@K**: Precisión de resultados
- **QPS**: Throughput (queries por segundo)
- **Latencia**: Tiempo por query
- **Uso de Memoria**: Eficiencia de recursos
- **Tiempo de Construcción**: Costo inicial

### Resultados Esperados
- **IVF-Flat**: Mejor recall, velocidad moderada
- **IVF-PQ**: Balance memoria/velocidad, recall ligeramente menor
- **CAGRA**: Máxima velocidad, recall aceptable para tiempo real

In [ ]:
print(f"IVF-PQ Build Time: {ivf_pq_build_time:.2f}s")
print(f"CAGRA Build Time: {cagra_build_time:.2f}s")
print(f"IVF-PQ Search Time: {ivf_pq_search_time:.4f}s ({n_queries / ivf_pq_search_time:.0f} QPS)")
print(f"CAGRA Search Time: {cagra_search_time:.4f}s ({n_queries / cagra_search_time:.0f} QPS)")

## 7. Visualizar Resultados de Rendimiento

Creamos gráficos para comparar algoritmos visualmente.

### Tipos de Gráficos
- **Recall vs QPS**: Trade-off precisión/velocidad
- **Uso de Memoria**: Eficiencia por algoritmo
- **Tiempo de Construcción**: Costo inicial
- **Escalabilidad**: Rendimiento vs tamaño del dataset

### Interpretación
- **Curvas de trade-off**: Encontrar punto óptimo para tu aplicación
- **Comparaciones absolutas**: Ver aceleración GPU vs CPU
- **Tendencias**: Cómo escala cada algoritmo

In [ ]:
!pip install faiss-gpu
import faiss

# FAISS IVF-PQ
quantizer = faiss.IndexFlatIP(dim)
index_faiss = faiss.IndexIVFPQ(quantizer, dim, 1024, pq_dim, 8)
index_faiss.train(dataset)
index_faiss.add(dataset)

start = time.time()
dist_faiss, neigh_faiss = index_faiss.search(queries, k)
faiss_search_time = time.time() - start

recall_faiss = recall_at_k(neigh_faiss, exact_neighbors, k)

print(f"FAISS IVF-PQ Search Time: {faiss_search_time:.4f}s")
print(f"FAISS Recall: {recall_faiss:.4f}")
print(f"CUVS IVF-PQ Recall: {recall_ivf_pq:.4f}")

## 8. Tuning Avanzado de Parámetros

Optimización específica para cada algoritmo.

### Estrategias por Algoritmo
- **IVF-PQ**: Optimizar n_subquantizers y n_bits para balance memoria/precisión
- **CAGRA**: Ajustar build_algo y search parameters para máxima velocidad
- **General**: Grid search o búsqueda bayesiana para parámetros óptimos

### Mejores Prácticas
- **Benchmarking**: Medir en datos representativos de producción
- **Validación cruzada**: Asegurar estabilidad de parámetros
- **Monitoreo**: Trackear rendimiento en producción

In [ ]:
algorithms = ['IVF-PQ', 'CAGRA', 'FAISS IVF-PQ']
recalls = [recall_ivf_pq, recall_cagra, recall_faiss]
times = [ivf_pq_search_time, cagra_search_time, faiss_search_time]

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.bar(algorithms, recalls)
plt.ylabel('Recall@10')
plt.title('Accuracy Comparison')

plt.subplot(1, 2, 2)
plt.bar(algorithms, times)
plt.ylabel('Search Time (s)')
plt.title('Performance Comparison')
plt.show()